<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Apple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Find the number of Apple product users (MacBook-Pro, iPhone 5s, iPad-air) and
the total number of users with any device, grouped by language
Output the language along with the total number of Apple users and users with
any device. Order the results by the number of total users in descending order.

In [9]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName("appleUsers").getOrCreate()

users_data = [
    (1, '2024-01-01 08:00:00', 101, 'English', '2024-01-05 10:00:00', 'Active'),
    (2, '2024-01-02 09:00:00', 102, 'Spanish', '2024-01-06 11:00:00', 'Inactive'),
    (3, '2024-01-03 10:00:00', 103, 'French', '2024-01-07 12:00:00', 'Active'),
    (4, '2024-01-04 11:00:00', 104, 'English', '2024-01-08 13:00:00', 'Active'),
    (5, '2024-01-05 12:00:00', 105, 'Spanish', '2024-01-09 14:00:00', 'Inactive')
]

events_data = [
    (1, '2024-01-05 14:00:00', 'Click', 'Login', 'USA', 'MacBook-Pro'),
    (2, '2024-01-06 15:00:00', 'View', 'Dashboard', 'Spain', 'iPhone 5s'),
    (3, '2024-01-07 16:00:00', 'Click', 'Logout', 'France', 'iPad-air'),
    (4, '2024-01-08 17:00:00', 'Purchase', 'Subscription', 'USA', 'Windows-Laptop'),
    (5, '2024-01-09 18:00:00', 'Click', 'Login', 'Spain', 'Android-Phone')
]

users_columns = ["user_id", "created_at", "company_id", "language", "activated_at", "state"]
events_columns = ["user_id", "occurred_at", "event_type", "event_name", "location", "device"]

users_df=spark.createDataFrame(users_data,users_columns)
events_df=spark.createDataFrame(events_data,events_columns)

users_df.show()

events_df.show()

apple_devices=['MacBook-Pro','iPhone 5s','iPad-air']

df1=(events_df.filter(col('device').isin(apple_devices))).join(users_df,"user_id","inner").groupBy('language').agg(countDistinct('user_id').alias('mac_users'))

df1.show()

df2=events_df.join(users_df,"user_id","inner").groupBy("language").agg(countDistinct("user_id").alias("total_users"))

df2.show()

df3=df1.join(df2,"language","outer").select("language",coalesce(col('mac_users'),lit(0)).alias("apple_users"),coalesce(col('total_users'),lit(0)).alias("total_users"))\
.orderBy(col('total_users').desc())

df3.show()


+-------+-------------------+----------+--------+-------------------+--------+
|user_id|         created_at|company_id|language|       activated_at|   state|
+-------+-------------------+----------+--------+-------------------+--------+
|      1|2024-01-01 08:00:00|       101| English|2024-01-05 10:00:00|  Active|
|      2|2024-01-02 09:00:00|       102| Spanish|2024-01-06 11:00:00|Inactive|
|      3|2024-01-03 10:00:00|       103|  French|2024-01-07 12:00:00|  Active|
|      4|2024-01-04 11:00:00|       104| English|2024-01-08 13:00:00|  Active|
|      5|2024-01-05 12:00:00|       105| Spanish|2024-01-09 14:00:00|Inactive|
+-------+-------------------+----------+--------+-------------------+--------+

+-------+-------------------+----------+------------+--------+--------------+
|user_id|        occurred_at|event_type|  event_name|location|        device|
+-------+-------------------+----------+------------+--------+--------------+
|      1|2024-01-05 14:00:00|     Click|       Login| 